# Cherry Swap Simulator

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
import ssl
import time
import math

In [ ]:
#disable ssl for cryptory API & virtualenv
ssl._create_default_https_context = ssl._create_unverified_context

Request Data from Compound.finance API

In [ ]:
cDaiAddress = '0xf5dce57282a584d2746faf1593d3121fcac444dc'
endTimestamp = math.floor(time.time())
startTimeStamp = endTimestamp - 6 * 30 * 24 * 60 * 60 #use the most recent 6 month's worth of data.
num_buckets = 24 * 30
requestURL = "https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=%s"% (cDaiAddress, startTimeStamp, endTimestamp, num_buckets)

response = requests.get(requestURL)

Convert Data into dataframe

In [ ]:
interestOverTime = pd.DataFrame.from_dict(response.json()['borrow_rates'])
interestOverTime.head()
interestOverTime['block_time'] = pd.to_datetime(interestOverTime['block_timestamp'],unit='s')
interestOverTime['rate_per_block'] = interestOverTime['rate'] / (4 * 60 * 24 * 365)
interestOverTime.head(20)

Plot Intrest rate over time

In [ ]:
data = [go.Scatter(
        x=interestOverTime['block_time'],
        y=interestOverTime['rate']
    )]

layout = go.Layout(
    title='Compound Interst Rate Over time',
    yaxis=dict(title='Lending Rate (%)'),
    xaxis=dict(title='Date'),
    template='plotly_white')

figure = go.Figure(data=data, layout=layout)

figure.show()

Random pools

In [ ]:
# random long and short pools
pool = pd.DataFrame(np.random.randint(0,1000,size=(1000, 2)), columns=list('LS'))
# total pool
pool['Total'] = pool.L + pool.S
# randomize whether test user is long or short
pool['isLong'] = np.random.randint(0,2,size=(1000, 1))
# random percentage test user holds in their pool
pool['pool_percentage'] = np.random.rand(1000,1)
# resulting holding of test user
pool['position'] =  pool.pool_percentage * (pool.isLong * pool.L + (1 - pool.isLong) * pool.S)
pool['total_percentage'] = pool.position / pool.Total
pool

In [ ]:
sim0 = pool
sim0['Total_float'] = pool.Total
i = 1
while interestOverTime['block_timestamp'][i] < interestOverTime['block_timestamp'][0] + 30 * 24 * 60 * 60:
    number_of_blocks = (interestOverTime['block_timestamp'][i] - interestOverTime['block_timestamp'][i-1]) / 15
    sim0['Total_float'] = sim0['Total_float'] * (1 + interestOverTime['rate_per_block'][i-1])**number_of_blocks
    i += 1
sim0['Total_fixed'] = sim0.Total * (1 + interestOverTime['rate_per_block'][0])**((interestOverTime['block_timestamp'][i] - interestOverTime['block_timestamp'][0]) / 15)
sim0['r_long'] = (sim0.S * (sim0['Total_float'] - sim0['Total_fixed'])) / sim0.Total**2
sim0['r_short'] = -1 * (sim0.L * (sim0['Total_float'] - sim0['Total_fixed'])) / sim0.Total**2
sim0['profit'] = sim0.position * (sim0.isLong * sim0.r_long + (1 - pool.isLong) * sim0.r_short)
sim0['payout'] = sim0.total_percentage * sim0.Total_float + sim0.profit
sim0['potential_payout'] = sim0.total_percentage * sim0.Total_float
sim0